In [1]:
import pandas as pd
from pandas import DataFrame
import math
import dataset
import main
import events
import os
import numpy as np

MINUTES = 20
SRC_DIR = 'dataset/bob_all_processed_mins'
OUT_DIR = f'dataset/b_parsed_{MINUTES}m'
os.makedirs(OUT_DIR, exist_ok=True)

columns = [
    'time',
    'month',
    'hour',
    't_i_1',
    't_i_2',
    't_i_3',
    't_i_4',
    't_o',
    'weight_kg',
    'h',
    't',
    'p',
    'queencell.next.dif',
    'feeding.next.dif',
    'honey.next.dif',
    'treatment.next.dif',
    'died.next.dif',
    'swarming.next.dif',
]

files = os.scandir(SRC_DIR)

In [2]:
for file in files:
    df = pd.read_csv(file.path, dtype={
        't_i_1': float,
        't_i_2': float,
        't_i_3': float,
        't_i_4': float,
        't_o': float,
        'weight_kg': float,
        "weight_delta": float,
        'numeric.time': float,
        'h': float,
        't': float,
        'p': float,
    }, low_memory=False)
    
    odf = pd.DataFrame(columns=columns)
    df = df.bfill()
    first = df.iloc[0]
    print(f"Processing {file.name}")
    no_queencell = first['queencell.next.dif'] == 'NA' or first['queencell.next.dif'] == '' or first['queencell.next.dif'] == None or math.isnan(first['queencell.next.dif'])
    no_feeding = first['feeding.next.dif'] == 'NA' or first['feeding.next.dif'] == '' or first['feeding.next.dif'] == None or math.isnan(first['feeding.next.dif'])
    no_honey = first['honey.next.dif'] == 'NA' or first['honey.next.dif'] == '' or first['honey.next.dif'] == None or math.isnan(first['honey.next.dif'])
    no_treatment = first['treatment.next.dif'] == 'NA' or first['treatment.next.dif'] == '' or first['treatment.next.dif'] == None or math.isnan(first['treatment.next.dif'])
    no_died = first['died.next.dif'] == 'NA' or first['died.next.dif'] == '' or first['died.next.dif'] == '0' or first['died.next.dif'] == None or math.isnan(first['died.next.dif'])
    no_swarming = first['swarming.next.dif'] == 'NA' or first['swarming.next.dif'] == '' or first['swarming.next.dif'] == None or math.isnan(first['swarming.next.dif'])
    
    if no_queencell == True and no_feeding == True and no_honey == True and no_treatment == True and no_died == True and no_swarming == True:
        print(f'Skipping {file.name}: Nothing there')
        continue
    print(f'Processing {file.name}: QC: {no_queencell} F: {no_feeding} H: {no_honey} T: {no_treatment} D: {no_died} S: {no_swarming}')

    for i, current in df.iterrows():
        # print(f"I: {i}/{len(df)}", end='\r')
        if i % MINUTES == 0:
            print(f"MI: {i}/{len(df)}", end='\r')

            if i > 0:
                slice = df.iloc[i-MINUTES:i]
                
                odf.loc[i // MINUTES] = {
                    'X.1': current['X.1'],
                    'month': current['month'],
                    'hour': current['hour'],

                    't_i_1': slice['t_i_1'].mean(),
                    't_i_2': slice['t_i_2'].mean(),
                    't_i_3': slice['t_i_3'].mean(),
                    't_i_4': slice['t_i_4'].mean(),

                    't_o': slice['t_o'].mean(),
                    'weight_kg': slice['weight_kg'].mean(),

                    'h': slice['h'].mean(),
                    't': slice['t'].mean(),
                    'p': slice['p'].mean(),
                    
                    'queencell.next.dif': current['queencell.next.dif'],
                    'feeding.next.dif': current['feeding.next.dif'],
                    'honey.next.dif': current['honey.next.dif'],
                    'treatment.next.dif': current['treatment.next.dif'],
                    'died.next.dif': current['died.next.dif'],
                    'swarming.next.dif': current['swarming.next.dif'],
                }
            else:
                odf.loc[i // MINUTES] = current[columns]

    
    odf = odf.bfill()

    odf['weight_kg'] = dataset.smooth_col(odf['weight_kg'])
    odf['t_i_1'] = dataset.smooth_col(odf['t_i_1'])
    odf['t_i_2'] = dataset.smooth_col(odf['t_i_2'])
    odf['t_i_3'] = dataset.smooth_col(odf['t_i_3'])
    odf['t_i_4'] = dataset.smooth_col(odf['t_i_4'])
    odf['t'] = dataset.smooth_col(odf['t'])
    odf['h'] = dataset.smooth_col(odf['h'])

    swarming_indexes = events.get_event_indexes(odf, 'swarming.next.dif')
    queencell_indexes = events.get_event_indexes(odf, 'queencell.next.dif')
    feeding_indexes = events.get_event_indexes(odf, 'feeding.next.dif')
    honey_indexes = events.get_event_indexes(odf, 'honey.next.dif')
    treatment_indexes = events.get_event_indexes(odf, 'treatment.next.dif')
    died_indexes = events.get_event_indexes(odf, 'died.next.dif')



    dataset.populate_column_by_index(odf, 'swarming', swarming_indexes)
    dataset.populate_column_by_index(odf, 'queencell', queencell_indexes)
    dataset.populate_column_by_index(odf, 'feeding', feeding_indexes)
    dataset.populate_column_by_index(odf, 'honey', honey_indexes)
    dataset.populate_column_by_index(odf, 'treatment', treatment_indexes)
    dataset.populate_column_by_index(odf, 'died', died_indexes)

    dataset.populate_delta(odf)
    dataset.populate_temp_delta(odf)
    dataset.populate_humidity_delta(odf)
    dataset.populate_normalized(odf, 'weight_delta_percent')
    dataset.populate_normalized(odf, 'weight_delta_diff')
    dataset.populate_scaled_weight(odf)

    odf.to_csv(f'{OUT_DIR}/{file.name}', index=False)

Processing 2019_62.csv
Processing 2019_62.csv: QC: True F: False H: False T: True D: True S: True


/Volumes/T7/PhD/HiveMonitor/dataset.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.0001604704411203854' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i + offset, 'weight_delta_percent'] = perc_delta


Processing 2020_87.csv
Processing 2020_87.csv: QC: False F: True H: False T: True D: True S: True


/Volumes/T7/PhD/HiveMonitor/dataset.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.00035513665334311586' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i + offset, 'weight_delta_percent'] = perc_delta


Processing 2020_57.csv
Processing 2020_57.csv: QC: True F: True H: True T: True D: False S: True


...

Processing 2022_67.csv
Skipping 2022_67.csv: Nothing there
Processing 2022_101.csv
Processing 2022_101.csv: QC: False F: True H: True T: True D: True S: False


/Volumes/T7/PhD/HiveMonitor/dataset.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00017240440629358878' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i + offset, 'weight_delta_percent'] = perc_delta


Processing 2022_109.csv
Skipping 2022_109.csv: Nothing there
Processing 2022_5.csv
Skipping 2022_5.csv: Nothing there
Processing 2022_152.csv
Processing 2022_152.csv: QC: True F: True H: False T: True D: True S: True


/Volumes/T7/PhD/HiveMonitor/dataset.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00016931123127377507' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i + offset, 'weight_delta_percent'] = perc_delta


Processing 2022_69.csv
Skipping 2022_69.csv: Nothing there
